In [24]:
import numpy as np
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import accuracy_score


# Download necessary NLTK data

In [2]:
# Download necessary NLTK data
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')


[nltk_data] Downloading package stopwords to /home/mukesh/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/mukesh/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/mukesh/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

# print the stopwords in English

In [3]:
# print the stopwords in English
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

# Load Dataset

In [4]:
#Load Dataset
news_dataset = pd.read_csv('train.csv')


In [5]:
news_dataset.shape

(20800, 5)

In [6]:
news_dataset.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [7]:
news_dataset.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [8]:
# Fill missing values
news_dataset = news_dataset.fillna('')


In [9]:
# Combine author, title, and text fields
news_dataset['content'] = news_dataset['author'] + ' ' + news_dataset['title'] + ' ' + news_dataset['text']


In [10]:
print(news_dataset['content'])

0        Darrell Lucus House Dem Aide: We Didn’t Even S...
1        Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo...
2        Consortiumnews.com Why the Truth Might Get You...
3        Jessica Purkiss 15 Civilians Killed In Single ...
4        Howard Portnoy Iranian woman jailed for fictio...
                               ...                        
20795    Jerome Hudson Rapper T.I.: Trump a ’Poster Chi...
20796    Benjamin Hoffman N.F.L. Playoffs: Schedule, Ma...
20797    Michael J. de la Merced and Rachel Abrams Macy...
20798    Alex Ansary NATO, Russia To Hold Parallel Exer...
20799    David Swanson What Keeps the F-35 Alive   Davi...
Name: content, Length: 20800, dtype: object


# Initialize the lemmatizer

In [11]:
# Initialize the lemmatizer
lemmatizer = WordNetLemmatizer()


In [12]:
# Function to preprocess and lemmatize the content
def lemmatize_content(content):
    lemmatized_content = re.sub('[^a-zA-Z]', ' ', content)
    lemmatized_content = lemmatized_content.lower()
    lemmatized_content = lemmatized_content.split()
    lemmatized_content = [lemmatizer.lemmatize(word) for word in lemmatized_content if word not in stopwords.words('english')]
    return ' '.join(lemmatized_content)

# Apply the lemmatization function
news_dataset['content'] = news_dataset['content'].apply(lemmatize_content)


# Prepare the data and labels

In [13]:
# Prepare the data and labels
X = news_dataset['content'].values
Y = news_dataset['label'].values


# Convert text data to numerical data using TF-IDF

In [14]:
# Convert text data to numerical data using TF-IDF
vectorizer = TfidfVectorizer(max_df=0.7, min_df=5, ngram_range=(1, 2), max_features=10000)
X = vectorizer.fit_transform(X)


# Split the dataset into training and testing sets

In [15]:
# Split the dataset into training and testing sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, stratify=Y, random_state=2)


In [26]:
from sklearn.linear_model import LogisticRegression
model= LogisticRegression()


In [27]:
# Set up hyperparameter grid for tuning
# Set up hyperparameter grid for tuning
param_grid = {
    'C': [0.1, 1, 10, 100],
    'penalty': ['l2'],  # L2 regularization is more common for logistic regression
    'solver': ['lbfgs', 'liblinear']
}



In [28]:
# Perform GridSearchCV for hyperparameter tuning
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, scoring='accuracy', n_jobs=-1)
grid_search.fit(X_train, Y_train)


ValueError: Invalid parameter 'max_depth' for estimator LogisticRegression(). Valid parameters are: ['C', 'class_weight', 'dual', 'fit_intercept', 'intercept_scaling', 'l1_ratio', 'max_iter', 'multi_class', 'n_jobs', 'penalty', 'random_state', 'solver', 'tol', 'verbose', 'warm_start'].

In [19]:
# Get the best model from the grid search
best_model = grid_search.best_estimator_


In [20]:
# Train the best model
best_model.fit(X_train, Y_train)


RandomForestClassifier(max_depth=20, min_samples_leaf=2, min_samples_split=5,
                       n_estimators=200)

In [21]:
# Evaluate the model on the training data
train_accuracy = accuracy_score(best_model.predict(X_train), Y_train)
print('Training accuracy:', train_accuracy)


Training accuracy: 0.9887620192307692


In [22]:
# Evaluate the model on the test data
test_accuracy = accuracy_score(best_model.predict(X_test), Y_test)
print('Test accuracy:', test_accuracy)


Test accuracy: 0.9591346153846154


In [23]:
# Making a Predictive System (example)
X_new = X_test[4]
prediction = best_model.predict(X_new)
print('Prediction:', prediction)

if prediction[0] == 0:
    print('The news is Real')
else:
    print('The news is Fake')

print('Actual label:', Y_test[4])


Prediction: [1]
The news is Fake
Actual label: 0
